# Backtesting a Strategy

In [ ]:
# !pip install import_ipynb --quiet
# !pip install anvil-uplink --quiet
# !pip install yfinance --quiet
# !pip install pandas_ta --quiet
# !pip install ipynb --quiet
# !pip install rpyc --quiet

In [ ]:
# # pull files from Github
# !git clone https://github.com/gmshroff/algostrats.git

In [ ]:
# %cd algostrats

In [1]:
import pandas as pd
import numpy as np
import import_ipynb
from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from feed_env.ipynb
importing Jupyter notebook from validation.ipynb


In [2]:
import warnings
warnings.simplefilter("ignore")

Import Strategy - replace with your own class or modify rulestrats

In [3]:
# # upload rulestrats.ipynb 
# from google.colab import files
# uploaded=files.upload()

In [4]:
# change to True if on colab
colab=False

In [5]:
from rulestrats import MomStrat,GapBet,AdaMomCMOADF, DTStrat

In [38]:
momStrat=MomStrat()

In [6]:
dtStrat = DTStrat()

In [39]:
adaMomStrat=AdaMomCMOADF()

In [8]:
# from ipynb.fs.defs.mlstrats import MLStrat,MLConfig,ConfigFields

In [9]:
# mlstrat=MLStrat()

Using BackFeed

In [7]:
data=pd.read_csv('./capvol100.csv')

In [8]:
tickers=list(data.iloc[0:10]['ticker'].values)

In [9]:
feed=BackFeed(tickers=tickers,nd=3,nw=1,synthetic=True)

In [13]:
# import pickle

In [14]:
# if not colab: 
#     with open('../../temp_data/btfeed.pickle','wb') as f: pickle.dump(feed,f)

In [15]:
# if not colab: 
#     with open('../../temp_data/btfeed.pickle','rb') as f: feed=pickle.load(f)

In [11]:
bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True,save_dfs=False)

In [14]:
bt.run_all(tickers=feed.tickers,model=dtStrat,verbose=False)

{'SYN':                         Close            datetime
2023-03-14 09:15:00  0.997601 2023-03-14 09:15:00
2023-03-14 09:20:00  0.994605 2023-03-14 09:20:00
2023-03-14 09:25:00  0.991620 2023-03-14 09:25:00
2023-03-14 09:30:00  0.988649 2023-03-14 09:30:00
2023-03-14 09:35:00  0.985698 2023-03-14 09:35:00
...                       ...                 ...
2023-03-14 15:10:00  0.990429 2023-03-14 15:10:00
2023-03-14 15:15:00  0.993410 2023-03-14 15:15:00
2023-03-14 15:20:00  0.996402 2023-03-14 15:20:00
2023-03-14 15:25:00  0.999400 2023-03-14 15:25:00
2023-03-15 09:15:00  1.000078 2023-03-15 09:15:00

[76 rows x 2 columns]}
{'SYN':                         Close            datetime
2023-03-14 09:15:00  0.997601 2023-03-14 09:15:00
2023-03-14 09:20:00  0.994605 2023-03-14 09:20:00
2023-03-14 09:25:00  0.991620 2023-03-14 09:25:00
2023-03-14 09:30:00  0.988649 2023-03-14 09:30:00
2023-03-14 09:35:00  0.985698 2023-03-14 09:35:00
...                       ...                 ...
2023-03-14

In [17]:
bt.run_all(tickers=feed.tickers,model=momStrat,verbose=False)

In [18]:
bt.run_all(tickers=feed.tickers,model=adaMomStrat,verbose=False)

In [20]:
bt.returns

{'ADANIGREEN.NS': 3.0481564368206544,
 'RELIANCE.NS': -0.7466132259453611,
 'TCS.NS': -1.7153482906017978,
 'MARUTI.NS': -2.18770868789832,
 'ADANITRANS.NS': -1.0655514667315829}

In [21]:
bt.results

{'ADANIGREEN.NS': {'15-Mar-2023': {'tot': 3.0481564368206544,
   'rew': [(5, 3.0481564368206544, 74)],
   'acts': [(1, 0, 0, 5)]}},
 'RELIANCE.NS': {'14-Mar-2023': {'tot': -0.7466132259453611,
   'rew': [(5, -0.7466132259453611, 74)],
   'acts': [(1, 0, 0, 5)]}},
 'TCS.NS': {'14-Mar-2023': {'tot': -1.7153482906017978,
   'rew': [(5, -1.7153482906017978, 74)],
   'acts': [(1, 0, 0, 5)]}},
 'MARUTI.NS': {'15-Mar-2023': {'tot': -2.18770868789832,
   'rew': [(5, -2.18770868789832, 74)],
   'acts': [(1, 0, 0, 5)]}},
 'ADANITRANS.NS': {'15-Mar-2023': {'tot': -0.4368648288952276,
   'rew': [(5, 2.860520955831519, 46),
    (46, -0.09999999999999999, 47),
    (47, -0.1, 48),
    (48, -0.1, 49),
    (49, -0.09999999999999999, 50),
    (50, -0.1, 51),
    (51, -0.09999999999999999, 52),
    (52, -0.09999999999999999, 53),
    (53, -0.09999999999999999, 54),
    (54, -0.09999999999999999, 55),
    (55, -0.09999999999999999, 56),
    (56, -0.09999999999999999, 57),
    (57, -0.09999999999999999, 58

In [22]:
from featfuncs import add_global_indices_feed

In [23]:
add_global_indices_feed(feed)

^NSEI: No data found for this date range, symbol may be delisted
^NYA: No data found for this date range, symbol may be delisted


In [24]:
feed.dates

array(['14-Mar-2023', '15-Mar-2023'], dtype=object)

In [25]:
gapStrat=GapBet()
gapStrat.gdata=feed.gdata

In [26]:
bt.run_all(tickers=feed.tickers,model=gapStrat,verbose=False)

KeyError: 'Close_^NYA'

In [ ]:
bt.returns

In [ ]:
bt.results

Using DataFeed

In [ ]:
# # upload kaggle.json
# from google.colab import files
# uploaded=files.upload()

In [ ]:
# !mkdir /root/.kaggle
# !mv ./kaggle.json /root/.kaggle/.
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# %mkdir data
# %cd data
# !kaggle datasets download -d gmshroff/marketdatafivemin
# !unzip marketdatafivemin.zip
# %cd ..

In [27]:
DATAFILE='augdata_16-Dec-2022_5m.csv'
if not colab: DATAPATH='../algodata/realdata/'
else: DATAPATH='./data/'
df=pd.read_csv(DATAPATH+DATAFILE)

In [28]:
dataFeed=DataFeed(tickers=list(df.ticker.unique()[0:50]),dfgiven=True,df=df)

In [29]:
bt=Backtest(dataFeed,tickers=dataFeed.tickers,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,deploy=True)

In [30]:
bt.run_all(tickers=dataFeed.tickers,model=momStrat)

In [31]:
bt.results

{'GEECEE.NS': {'05-Dec-2022': {'tot': -1.3090116279069826,
   'rew': [(0, -0.1, 1),
    (1, -0.09999999999999999, 2),
    (2, -0.1, 3),
    (3, -0.1, 4),
    (4, -0.1, 5),
    (5, -0.09999999999999999, 6),
    (6, -0.10000000000000002, 7),
    (7, -0.1, 8),
    (8, -0.1, 9),
    (9, -0.1, 10),
    (10, -0.1, 11),
    (11, -0.1, 12),
    (15, -0.10000000000000002, 16),
    (18, 0.19098837209301764, 40),
    (40, -0.1, 41),
    (41, -0.1, 42)],
   'acts': [(-1, 0, 0, 0),
    (-1, 0, 0, 1),
    (-1, 0, 0, 2),
    (-1, 0, 0, 3),
    (-1, 0, 0, 4),
    (-1, 0, 0, 5),
    (-1, 0, 0, 6),
    (-1, 0, 0, 7),
    (-1, 0, 0, 8),
    (-1, 0, 0, 9),
    (-1, 0, 0, 10),
    (-1, 0, 0, 11),
    (-1, 0, 0, 15),
    (-1, 0, 0, 18),
    (1, 0, 0, 40),
    (1, 0, 0, 41)]}},
 'VSTIND.NS': {'28-Nov-2022': {'tot': -6.154588074739631,
   'rew': [(0, -0.1, 1),
    (1, -0.09999999999999999, 2),
    (2, -0.1, 3),
    (3, -0.1, 4),
    (4, -0.09999999999999999, 5),
    (5, -0.1, 6),
    (6, -0.1, 7),
    (7, -0.

In [32]:
bt.returns

{'GEECEE.NS': -1.3090116279069826,
 'VSTIND.NS': -6.154588074739631,
 'DCM.NS': -34.383941692907435,
 'ZEEMEDIA.NS': -13.925966514330128,
 'CYIENT.NS': -0.6744230769230795,
 'J&KBANK.NS': -27.635214799712124,
 'JPOLYINVST.NS': -21.567921031094276,
 'ZEELEARN.NS': -5.1999999999999975,
 'MBECL.NS': -31.650409569330492,
 'ALMONDZ.NS': -0.0939089567765039,
 'PCJEWELLER.NS': -11.944536486762985,
 'JINDWORLD.NS': -8.827991960234074,
 'CINELINE.NS': -11.498470477784956,
 'SABTN.NS': 0.0,
 'INDIANCARD.NS': -2.2972359812962693,
 'KPRMILL.NS': -2.5367271385678727,
 'CREATIVEYE.NS': 0.0}

Validating remote strategy functions - IGNORE FOR NOW

In [ ]:
try:
    with open('/Users/a112956/MyCode/algo_fin_root/algo_fin_src/anvilcode.txt','r') as f: 
        CODE=f.read()
except:
    CODE=''

In [ ]:
import anvil.server
anvil.server.connect(CODE)

In [ ]:
# Note - first do a backtest using a local strategy; this will also update the feed with
# required features. Only then do remote validation. Else do the following first:
from featfuncs import add_addl_features_feed,add_sym_feature_feed
add_addl_features_feed(feed,tickers=feed.tickers)
add_sym_feature_feed(feed,tickers=feed.tickers)

In [ ]:
dfD={feed.tickers[i]:feed.getDataN(feed.tickers[i],feed.dates[0]) for i in range(4)}

In [ ]:
rpcname='s0_at_a_dot_edu'

In [ ]:
retval=Validate.remote_check_entry_batch(dfD,rpcname)

In [ ]:
list(momStrat.check_entry_batch(dfD))

In [ ]:
Validate(verbose=False).validate(retval=list(momStrat.check_entry_batch(dfD)),dfD=dfD)

In [ ]:
retval=Validate(feed).validate(retval=retval,dfD=dfD)

In [ ]:
retval1=retval.copy()
retval1[0]={'MARUTI.NS': 'five', 'KOTAKBANK.NS': 5, 'NESTLEIND.NS': 5, 'COALINDIA.NS': 5}

In [ ]:
Validate(feed).validate(retval=retval,dfD=dfD)

Backtesting remote strategy

In [ ]:
try:
    with open('/Users/a112956/MyCode/algo_fin_root/algo_fin_src/anvilcode.txt','r') as f: 
        CODE=f.read()
except:
    CODE=''

In [ ]:
import anvil.server
anvil.server.connect(CODE)

In [ ]:
bt=Backtest(feed,tickers=feed.tickers,add_features=True,target=.05,stop=.01,txcost=0.001,
            loc_exit=True,scan=True,topk=3,remote=True,
            data_cols=momStrat.data_cols,rpcname='s0_at_a_dot_edu')

In [ ]:
bt.run_all(tickers=feed.tickers)

In [ ]:
bt.results

In [ ]:
bt.returns

Experiments